# Examine repeatability of CWFS measurements when 10 datasets are taken consequetively
## This is for data taken on 2021-07-07 at 03:01 CLT (images 462-499) 
### It first finds the pairs of CWFS images in the EFD
### It then fits the data a reports the zernikes

In [ ]:
import asyncio 
import matplotlib

import numpy as np
import pandas as pd

from matplotlib import pylab as plt
from astropy.time import Time, TimeDelta

from lsst_efd_client import EfdClient, resample, rendezvous_dataframes

%matplotlib inline

In [ ]:
#efd_client = EfdClient('summit_efd')
efd_client = EfdClient('ldf_stable_efd') 

### Find CWFS pairs
Query for all the `endReadout` events on the timespan of the night.

In [ ]:
date='20210707'
test='cwfs_repeatability'

In [ ]:
run='initial' # only partial as a timeout and weird slew happened

if run == 'initial':
    # times from when images were taken, 
    t1_set = Time("2021-07-08T07:15:34", format='isot', scale='tai')
    t2_set = Time("2021-07-08T08:01:04", format='isot', scale='tai')


In [ ]:
t1_set

In [ ]:
end_readout = await efd_client.select_time_series("lsst.sal.ATCamera.logevent_endReadout", 
                                           ["imageName", "requestedExposureTime", "additionalKeys",
                                            "additionalValues","timestampAcquisitionStart","timestampEndOfReadout"], t1_set, t2_set)

In [ ]:
end_readout

In [ ]:
base0 = await efd_client.select_time_series("lsst.sal.Script.logevent_logMessage", 
                                           ["message","level"], t1_set, t2_set)

In [ ]:
# query = f'SELECT "expectedAzimuth", "expectedElevation", "measuredAzimuth", "measuredElevation", "measuredRotator" '\
#         f'FROM "efd"."autogen"."lsst.sal.ATPtg.logevent_pointData" WHERE {timestamp}'
base0 = await efd_client.select_time_series("lsst.sal.ATPtg.logevent_pointData", 
                                           ["measuredElevation","expectedElevation"], t1_set, t2_set)

In [ ]:
# print(base0.to_string())

In [ ]:
# Find the start/end sequences and trim the bad ones
# base = base0[(base0.message.str.find(start_log_msg) != -1) | (base0.message.str.find(finish_log_msg) != -1)]

In [ ]:
# Drop false starts
# if run == 'initial':
#     time_reg = '2021-07-09 02:41:02.488000+00:00'
#     ind=base.index.get_loc(time_reg, method='nearest')
#     base.drop(base.index[ind])

In [ ]:
base=base0

In [ ]:
base

Now match each entry. For each `i` item with `intra` in the name, there must be an `i+1` with `extra` otherwise it is not a pair. 
A pair also has the same groupID
The image before the pair is an in-focus image.

In [ ]:
intra_images = []
extra_images = []
in_focus_images = []
intra_times = []
extra_times = []
in_focus_times = []
intra_exptimes = []
extra_exptimes = []
in_focus_exptimes = []

df=pd.DataFrame()

i = 0
npairs = 0
nmiss = 0

while i < len(end_readout)-1:
    intra = end_readout['imageName'][i]
    extra = end_readout['imageName'][i+1]
    in_focus = end_readout['imageName'][i+2]
    
    #skip known bad files
    if intra == 'AT_O_20200218_000179' and extra == 'AT_O_20200218_000180':
        i+=2
        continue
    # this is horrible, but looks for sequence as it was taken
    # 2s in focus object, then a few CWFS frames, then 5s in focus object
    # finds the in-focus image by seeing if an OBJECT was taken right before the pair
    # using a colon to separate values causes issues because there are colons in the timestamp!
    group_id_cwfs_in=(end_readout.additionalValues[i])[0:25]
    imgtype_cwfs_in=(end_readout.additionalValues[i])[-6:]
    group_id_cwfs_out=(end_readout.additionalValues[i+1])[0:25] 
    imgtype_cwfs_out=(end_readout.additionalValues[i+1])[-6:]
    group_id_5s=(end_readout.additionalValues[i+2])[0:25]
    imgtype_5s=(end_readout.additionalValues[i+2])[-6:]

    if ((group_id_cwfs_in == group_id_cwfs_out) and 
        (imgtype_5s == 'OBJECT') and
       imgtype_cwfs_in == 'NGTEST'):
        
        print(f"Got a pair: {intra} x {extra}, with in-focus of {in_focus}")
        df_tmp=pd.DataFrame({'inFocus5s':end_readout['imageName'][i+2],
                             'intra':end_readout['imageName'][i],
                             'extra':end_readout['imageName'][i+1],
                             # Need times during cwfs for telescope position
                             'inFocusExpTime':end_readout['requestedExposureTime'][i+2],
                             'inFocustimestampEndOfReadout':end_readout['timestampEndOfReadout'][i+2],
                             'intraExtratimestampAcquisitionStart':end_readout['timestampAcquisitionStart'][i],
                             'intraExtratimestampEndOfReadout':end_readout['timestampEndOfReadout'][i+1],
                            }, index=[end_readout.index[i+2]])
        df=df.append(df_tmp)
        i+=2
        npairs+=1
    else:
#         print(f"No Match: {intra} x {extra}")
        nmiss+=1
        i+=1

print(f"Got {npairs} pairs and {nmiss} misses.")

In [ ]:
df

In [ ]:
# create new dataframe with new values of interest and we'll join them post-facto
df_offsets=pd.DataFrame()
# Populate the data structure from the pairs found above
for i in range(len(df.index)):
    
    # Determine time stamps for searching for metadata
    # include ability to correct for TAI if required, but set to zero for the moment

#    t1 = Time(in_focus_times[i], scale='tai') - TimeDelta(in_focus_exptimes[i], format='sec', scale='tai')
#    t2 = Time(extra_times[i], scale='tai') - TimeDelta(2., format='sec', scale='tai')
    
    # want time during CWFS sensing for telescope position
    t1 = Time(df['intraExtratimestampAcquisitionStart'][i], format='unix_tai')
    t2 = Time(df['intraExtratimestampEndOfReadout'][i],format='unix_tai')
    
    azel = await efd_client.select_time_series("lsst.sal.ATMCS.mount_AzEl_Encoders", 
                                               ["elevationCalculatedAngle99", "azimuthCalculatedAngle99"], t1, t2)
    # mount reporting incorrect timestamp
    azel.index=azel.index+pd.tseries.offsets.DateOffset(seconds=-37)
    
    rotator = await efd_client.select_time_series("lsst.sal.ATMCS.mount_Nasmyth_Encoders",
                                                  ["nasmyth2CalculatedAngle99"], t1, t2)
    # mount reporting incorrect timestamp
    rotator.index=rotator.index+pd.tseries.offsets.DateOffset(seconds=-37)

    m1_pressure = await efd_client.select_time_series("lsst.sal.ATPneumatics.m1AirPressure",
                                                  ["pressure"], t1, t2)
    # mount reporting incorrect timestamp
    m1_pressure.index=m1_pressure.index+pd.tseries.offsets.DateOffset(seconds=-37)
    
    # want time during long exposure for hexapod position (or basically just not the CWFS data)
    t2_hex = Time(df['inFocustimestampEndOfReadout'][i], format='unix_tai')
    t1_hex = t1-TimeDelta(df['inFocusExpTime'][i], format='sec', scale='tai') # this is subtraction, so before endOfReadout event above
    hexapod_vals = await efd_client.select_time_series("lsst.sal.ATHexapod.positionStatus", 
                                       ["reportedPosition0", "reportedPosition1", "reportedPosition2",
                                       "reportedPosition3", "reportedPosition4", "reportedPosition5"], t1_hex , t2_hex)

# For offsets we want to find the offsets between the start of the set and the beginning of the in-focus image, but the end works too
#     cmd_offset = await efd_client.select_time_series("lsst.sal.ATAOS.command_offset",
#                                                  ["u", "v", "w", "x", "y", "z"], offset_start , Time(df['inFocustimestampEndOfReadout'][i], format='unix_tai'))
    
    # to use the dataframe.between_time(), convert astropy Time object, to numpy time object, to pandas time object, and get the datetime.time
#     time1=pd.to_datetime(t1_set.to_datetime()).time()
#     time2=pd.to_datetime((Time(df['inFocustimestampEndOfReadout'][i],format='unix_tai')).to_datetime()).time()

    df_tmp=pd.DataFrame({'rot_pos':np.mean(rotator['nasmyth2CalculatedAngle99']),
                     'el':np.mean(azel['elevationCalculatedAngle99']),
                     'az':np.mean(azel['azimuthCalculatedAngle99']),
                     'x':hexapod_vals['reportedPosition0'].median(),
                     'y':hexapod_vals['reportedPosition1'].median(),
                     'z':hexapod_vals['reportedPosition2'].median(),
                     'u':hexapod_vals['reportedPosition3'].median(),
                     'v':hexapod_vals['reportedPosition4'].median(),
                     'w':hexapod_vals['reportedPosition5'].median(),
                     'm1': np.mean(m1_pressure['pressure']),
#                      'hexXoffset': cmd_offset['x'].sum(),
#                      'hexYoffset': cmd_offset['y'].sum(),
#                      'hexUoffset': cmd_offset['u'].sum(),
#                      'hexVoffset': cmd_offset['v'].sum(),
#                      'hexZoffset': cmd_offset['z'].sum(),
                        },
                     index=[df.index[i]])
    df_offsets=df_offsets.append(df_tmp)

In [ ]:
# Join the two dataframes to create a single one
df=df.join(df_offsets, lsuffix='_caller', rsuffix='_other')

In [ ]:
df

In [ ]:
# out = rendezvous_dataframes(end_readout, cmd_offset, direction='backward', tolerance=pd.Timedelta(days=1))

In [ ]:
filename="data/"+date+"_"+run+"_metadata.csv"
df.to_csv(filename)

# Now reduce the data for each pair to get the zernikes from fitting
#### You can start here if the top bit has already run 

In [ ]:
def atcs_get_bore_sight_angle(elevation_angle, nasmyth_angle):
    # modified from atcs.py
    # instrument on nasymth2, so
    parity_x = -1
    bore_sight_angle = elevation_angle + parity_x * nasmyth_angle + 90.0
    return bore_sight_angle

In [ ]:
import sys
import asyncio
import logging
import numpy as np
from lsst.ts.externalscripts.auxtel.latiss_cwfs_align import LatissCWFSAlign
import time
import os

In [ ]:
# Read in the file (written using code above)
filename="data/"+date+"_"+run+"_metadata.csv"
df2 = pd.read_csv(filename, index_col=0)
df2.index=pd.to_datetime(df2.index)

In [ ]:
df2

In [ ]:
# Add all the zernike terms from the fitting
# this creates columns of NaNs and the loop below populates them
df2[['zern_defocus_nm', 'zern_astig_oblique_nm','zern_astig_vertical_nm',
    'zern_coma_vertical_nm', 'zern_coma_horizontal_nm',
    'zern_trefoil_vertical_nm', 'zern_trefoil_oblique_nm','zern_spherical_nm' ]] = np.nan

In [ ]:
# this just needs to be set, but is not actually used
os.environ['LSST_DDS_DOMAIN'] = 'junk'

In [ ]:
# alignment script needs to have remotes set to False! Otherwise it'll try to command the hexapod!
script = LatissCWFSAlign(index=1, remotes=False)

In [ ]:
# define the location of the butler repo
script.dataPath='/project/shared/auxTel/'

In [ ]:
def get_visitID_from_filename(filename):
    # Expects AT_O_20200218_000167.fits
    # parse out visitID from filename - this is highly annoying
    tmp=filename.split('_')
    prefix=tmp[2] # dayobs without the dashes

    # Don't remember why I used int here... whitespace? 
    # surely fixable but bigger fish.
    suffix='{:05d}'.format(int(tmp[3].split('.')[0])) # SEQNUM, but need to trim extra 0 in obsid
    visitID = int((prefix+suffix))
    #print(visitID)
    return visitID

In [ ]:
# bin images when performing fits?
script.binning=2

In [ ]:
for n in range(len(df2)):
    # see tstn-015 and example notebook on running latiss_align_cwfs script
    script.intra_visit_id = get_visitID_from_filename(df2['intra'].iloc[n])
    script.extra_visit_id = get_visitID_from_filename(df2['extra'].iloc[n])
    script.angle = 90-atcs_get_bore_sight_angle(df2['el'].iloc[n], df2['rot_pos'].iloc[n])

    start_time=time.time()
    await script.run_cwfs()
    end_time=time.time()
    print('WFE fitting for visitIDs {0} and {1} took {2:0.3f} seconds'.format(script.intra_visit_id, script.extra_visit_id,end_time-start_time)) # 56.7s

    # Display fitting results?
    if (False):
        # plot zernikes
        x = np.arange(9)+4
        plt.plot(x, script.algo.zer4UpNm[:9], 'o-', label=f'{script.dz}')
        xlim = plt.xlim()
        plt.plot(np.arange(15), np.zeros(15)+50, 'b--')
        plt.plot(np.arange(15), np.zeros(15)-50, 'b--')
        plt.xlim(xlim)
        plt.ylabel("Zernike coeff (nm)")
        plt.xlabel("Zernike index")
        plt.grid()
        plt.legend()
        
    if (True):
        # plot image and mask
        fig1 = plt.figure(2, figsize=(12,8))
        ax11 = fig1.add_subplot(121)
        ax11.set_title("defocus 0.8 - intra")
        ax11.imshow(script.I1[0].image0)
        ax11.contour(script.algo.pMask) 
        ax12 = fig1.add_subplot(122)
        ax12.set_title("defocus 0.8 - extra")
        ax12.imshow(script.I2[0].image0)
        ax12.contour(script.algo.pMask) 
        
    # Put results into data structure 
    df2.loc[df2.index[n],('zern_defocus_nm')] = script.algo.zer4UpNm[0]
    df2.loc[df2.index[n],('zern_astig_oblique_nm')] = script.algo.zer4UpNm[1] # once labeled x-astigmatism
    df2.loc[df2.index[n],('zern_astig_vertical_nm')] = script.algo.zer4UpNm[2]
    df2.loc[df2.index[n],('zern_coma_vertical_nm')] = script.algo.zer4UpNm[3] # formerly x-coma
    df2.loc[df2.index[n],('zern_coma_horizontal_nm')]= script.algo.zer4UpNm[4] # formerly y-coma
    df2.loc[df2.index[n],('zern_trefoil_vertical_nm')] = script.algo.zer4UpNm[5]  # once labeled xtrefoil
    df2.loc[df2.index[n],('zern_trefoil_oblique_nm')] = script.algo.zer4UpNm[6]
    df2.loc[df2.index[n],('zern_spherical_nm')] = script.algo.zer4UpNm[7]

In [ ]:
#df2=df2.rename(columns={'zern_xtrefoil_nm': 'zern_trefoil_vertical_nm', 'zern_ytrefoil_nm': 'zern_trefoil_oblique_nm'})

In [ ]:
# df2=df2.rename(columns={'zern_xastig_nm': 'zern_astig_oblique_nm', 'zern_yastig_nm': 'zern_astig_vertical_nm'})

In [ ]:
# df2=df2.rename(columns={'zern_ycoma_nm': 'zern_coma_horizontal_nm', 'zern_xcoma_nm': 'zern_coma_vertical_nm'})

In [ ]:
df2.keys()

In [ ]:
#write to CSV file
filename="data/"+date+"_"+run+"_data_with_WFE_in_zerns.csv"

In [ ]:
df2.to_csv(filename)

## Get M1 mirror data and fit a plane

In [ ]:
pmd = await efd_client.select_time_series("lsst.sal.PMD.position", ["position0", "position1", "position2", "position3", "position4"], t1_set, t2_set)

In [ ]:
# Read in the file (written using code above)
filename="data/"+date+"_"+run+"_data_with_WFE_in_zerns.csv"
df3 = pd.read_csv(filename, index_col=0)
df3.index=pd.to_datetime(df3.index)

In [ ]:
df4=rendezvous_dataframes(df3, pmd)

In [ ]:
position0_offset = (df4.position0[0])
position1_offset = (df4.position1[0])
position2_offset = (df4.position2[0])
position3_offset = (df4.position3[0])
position4_offset = (df4.position4[0])

In [ ]:
from scipy import linalg
arr_len = len(df4.position0)
coeff_arr = np.zeros((arr_len,3))
theta_arr = np.zeros((arr_len))
phi_arr = np.zeros((arr_len))
piston_arr = np.zeros((arr_len))
for i in np.arange(arr_len):

    # X, Y, Z - measured from solidmodel
    set2=np.array((  41.0, 468.0, (df4.position2[i]-position2_offset)))
    set3=np.array(( 384.0,-269.0, (df4.position3[i]-position3_offset)))
    set4=np.array((-425.0,-198.0, (df4.position4[i]-position4_offset)))

    # Vector PQ crossed with Vector PR
    normal = np.cross(set3-set2,set4-set2) # gives a,b,c
    #print(f'normal is {normal}')
#     theta_arr[i] = (np.pi/2 + np.arctan2(normal[2],normal[0])) * 206265 # arcsec
#     phi_arr[i] = (np.pi/2 + np.arctan2(normal[2],normal[1])) * 206265   # arcsec
#     piston_arr[i] = normal[2]
    
#     => a * (x - x0) + b * (y - y0) + c * (z - z0) = 0.
# => a * x - a * x0 + b * y - b * y0 + c * z - c * z0 = 0.
# => a * x + b * y + c * z + (- a * x0 - b * y0 - c * z0) = 0. # D is the last terms
    D= -normal[0]*set2[0] - normal[1]*set2[1] - normal[2]*set2[2]  # Constant in plane equation
    # equation 
    
    phi_from_normal = (np.pi/2+np.arctan2(normal[2], normal[1])) * 206265
    theta_from_normal = (np.pi/2+np.arctan2(normal[2], normal[0])) * 206265
    # find z at the origin to represent piston
    Z_origin = -D/normal[2]
    
    # Measure rotation about the Y-axis (perpendicular to elevation)
    # So this is TIP and should result in motion in azimuth
    # get slope by looking at Y=0, X=400
    x_pt=400; y_pt=0
    #Z_at_x_pt= C[0]*x_pt + C[1]*0.0 + C[2]
    Z_at_x_pt= (-D - normal[0]*x_pt - normal[1]*0.0)/normal[2]
    theta = np.arctan2(Z_at_x_pt-Z_origin, x_pt) * 206265 # arcsec
        
    # Measure rotation about the X-axis (aligned to elevation)
    # this is TILT and should result in motion in elevation
    # get slope by looking at Y=0, X=400    
    x_pt=0; y_pt=400
    Z_at_y_pt= (-D - normal[0]*x_pt - normal[1]*y_pt)/normal[2]
    phi = np.arctan2(Z_at_y_pt-Z_origin, y_pt) * 206265 # arcsec

    theta_arr[i] = theta # arcsec
    phi_arr[i] = phi    # arcsec
    piston_arr[i] = Z_origin
    
    print(f'theta_from_normal is {theta_from_normal:0.2f}, phi_from_normal is {phi_from_normal:0.2f}')
    print(f'theta_arr is {theta_arr[i]:0.2f} [arcsec], phi_arr is {phi_arr[i]:0.2f} [arcsec]')
#     if i == 1:
#         break

In [ ]:
df4

In [ ]:
df4['m1_tip']=theta_arr
df4['m1_tilt']=phi_arr
df4['m1_piston']=piston_arr

df4['m1_y_pos']=(df4.position0-position0_offset)*np.cos(10*np.pi/180)
df4['m1_x_pos']=(df4.position1-position1_offset)*np.cos(10*np.pi/180)

In [ ]:
#write to CSV file
filename="data/"+date+"_"+run+"_data_with_WFE_and_m1_pos.csv"

In [ ]:
df4.to_csv(filename)

#### Plot the M1 data

In [ ]:
# Read in the file (written using code above)
filename="data/"+date+"_"+run+"_data_with_WFE_and_m1_pos.csv"
df4 = pd.read_csv(filename, index_col=0)
df4.index=pd.to_datetime(df4.index)

In [ ]:
df4.keys()

In [ ]:
fig_height=5
fig_width=11

In [ ]:
%matplotlib inline
nwide=3; nhigh=3
fig, (row1,row2,row3) = plt.subplots(nhigh, nwide, figsize=(nwide+fig_width, nhigh*fig_height))
fig.suptitle('Rows are')
xvals = (df4.m1_x_pos) # um
yvals = df4.el

row1[0].plot(xvals, yvals, 'o-')
row1[0].set_ylabel('Elevation [deg]')
row1[0].set_xlabel('M1 X-position [mm]')

xvals=df4.y
row1[1].plot(xvals,yvals, '.-')
row1[1].set_xlabel('Hexapod Y-position [mm]')

xvals = (df4.m1_tip) # um
row1[2].plot(xvals, yvals, 'o-')
row1[2].set_xlabel('M1 tip [arcsec]')

xvals = (df4.m1_y_pos) # mm
row2[0].plot(xvals, yvals, 'o-')
row2[0].set_ylabel('Elevation [deg]')
row2[0].set_xlabel('M1 X-position [mm]')

xvals=df4.x
row2[1].plot(xvals,yvals, '.-')
row2[1].set_xlabel('Hexapod X-position [mm]')

xvals = (df4.m1_tilt) # um
row2[2].plot(xvals, yvals, 'o-')
row2[2].set_xlabel('M1 tilt [arcsec]')

xvals = (df4.m1_piston) # mm
row3[0].plot(xvals, yvals, 'o-')
row3[0].set_ylabel('Elevation [deg]')
row3[0].set_xlabel('M1 Z-position [mm]')

xvals=df4.z
row3[1].plot(xvals,yvals, '.-')
row3[1].set_xlabel('Hexapod Z-position [mm]')

In [ ]:
%matplotlib inline
nwide=4; nhigh=3
fig, (row1,row2,row3) = plt.subplots(nhigh, nwide, figsize=(nwide+fig_width, nhigh*fig_height))
fig.suptitle('Rows are')
xvals = (df4.m1_x_pos) # um
yvals = df4.el

row1[0].plot(xvals, yvals, 'o-')
row1[0].set_ylabel('Elevation [deg]')
row1[0].set_xlabel('M1 X-position [mm]')

xvals=df4.y
row1[1].plot(xvals,yvals, '.-')
row1[1].set_xlabel('Hexapod Y-position [mm]')
row1[1].annotate(f'$\sigma$ = {np.std(xvals):0.3f}',(0.7,0.9), xycoords='axes fraction',)

xvals = (df4.m1_tip) # um   - Tip should mean ~azimuth motion
row1[2].plot(xvals, yvals, 'o-')
row1[2].set_xlabel('M1 tip [arcsec]')

xvals = (df4.zern_trefoil_vertical_nm) # vertical
row1[3].plot(xvals, yvals, 'o-')
row1[3].set_xlabel('Trefoil Vertical[nm]')

xvals = (df4.m1_y_pos) # mm
row2[0].plot(xvals, yvals, 'o-')
row2[0].set_ylabel('Elevation [deg]')
row2[0].set_xlabel('M1 X-position [mm]')
row2[0].annotate(f'$\sigma$ = {np.std(xvals):0.3f}',(0.7,0.9), xycoords='axes fraction',)

xvals=df4.x
row2[1].plot(xvals,yvals, '.-')
row2[1].set_xlabel('Hexapod X-position [mm]')
row2[1].annotate(f'$\sigma$ = {np.std(xvals):0.3f}',(0.7,0.9), xycoords='axes fraction',)

xvals = (df4.m1_tilt) # um
row2[2].plot(xvals, yvals, 'o-')
row2[2].set_xlabel('M1 tilt [arcsec]')
row2[2].annotate(f'$\sigma$ = {np.std(xvals):0.3f}',(0.7,0.9), xycoords='axes fraction',)

xvals = (df4.zern_trefoil_oblique_nm) # oblique
row2[3].plot(xvals, yvals, 'o-')
row2[3].set_xlabel('Trefoil Oblique [nm]')

xvals = (df4.m1_piston) # mm
row3[0].plot(xvals, yvals, 'o-')
row3[0].set_ylabel('Elevation [deg]')
row3[0].set_xlabel('M1 Z-position [mm]')

xvals=df4.z
row3[1].plot(xvals,yvals, '.-')
row3[1].set_xlabel('Hexapod Z-position [mm]')


In [ ]:
print(np.sqrt(df4.zern_defocus_nm**2+df4.zern_spherical_nm**2+df4.zern_coma_vertical_nm**2+df4.zern_trefoil_vertical_nm**2+df4.zern_astig_vertical_nm**2+df4.zern_coma_horizontal_nm**2+df4.zern_trefoil_oblique_nm**2+df4.zern_astig_oblique_nm**2))

In [ ]:
%matplotlib inline
nwide=3; nhigh=3
fig, (row1,row2,row3) = plt.subplots(nhigh, nwide, figsize=(nwide+fig_width, nhigh*fig_height))
fig.suptitle('Rows are')
xvals = (df4.zern_defocus_nm) # nm
yvals = df4.el

row1[0].plot(xvals, yvals, 'o-')
row1[0].set_ylabel('Elevation [deg]')
row1[0].set_xlabel('Defocus [nm]')
row1[0].annotate(f'$\sigma$ = {np.std(xvals):0.1f}',(0.75,0.9), xycoords='axes fraction',)

xvals = (df4.zern_spherical_nm) # nm
row1[1].plot(xvals, yvals, 'o-')
row1[1].set_xlabel('Spherical [nm]')
row1[1].annotate(f'$\sigma$ = {np.std(xvals):0.1f}',(0.75,0.9), xycoords='axes fraction',)

xvals = (df4.zern_astig_vertical_nm) # nm
row2[0].plot(xvals, yvals, 'o-')
row2[0].set_ylabel('Elevation [deg]')
row2[0].set_xlabel('Vertical Astigmatism [nm]')
row2[0].annotate(f'$\sigma$ = {np.std(xvals):0.1f}',(0.75,0.9), xycoords='axes fraction',)

xvals = (df4.zern_coma_vertical_nm) # nm
row2[1].plot(xvals, yvals, 'o-')
row2[1].set_xlabel('Vertical Coma [nm]')
row2[1].annotate(f'$\sigma$ = {np.std(xvals):0.1f}',(0.75,0.9), xycoords='axes fraction',)

xvals = (df4.zern_trefoil_vertical_nm) # nm
row2[2].plot(xvals, yvals, 'o-')
row2[2].set_xlabel('Vertical Trefoil [nm]')
row2[2].annotate(f'$\sigma$ = {np.std(xvals):0.1f}',(0.75,0.9), xycoords='axes fraction',)

xvals = (df4.zern_astig_oblique_nm) # nm
row3[0].plot(xvals, yvals, 'o-')
row3[0].set_ylabel('Elevation [deg]')
row3[0].set_xlabel('Oblique Astigmatism [nm]')
row3[0].annotate(f'$\sigma$ = {np.std(xvals):0.1f}',(0.75,0.9), xycoords='axes fraction',)

xvals = (df4.zern_coma_horizontal_nm) # nm
row3[1].plot(xvals, yvals, 'o-')
row3[1].set_xlabel('Horizontal Coma [nm]')
row3[1].annotate(f'$\sigma$ = {np.std(xvals):0.1f}',(0.75,0.9), xycoords='axes fraction',)

xvals = (df4.zern_trefoil_oblique_nm) # nm
row3[2].plot(xvals, yvals, 'o-')
row3[2].set_xlabel('Oblique Trefoil [nm]')
row3[2].annotate(f'$\sigma$ = {np.std(xvals):0.1f}',(0.75,0.9), xycoords='axes fraction',)

In [ ]:
runs = ['iter3','iter4']
sym= ['.-','x-','triangle_up-']
colors=['red','blue','green']
df={}

fig_height=5
fig_width=10

%matplotlib inline
from matplotlib import colors

nwide=4; nhigh=3
fig, (row1,row2,row3) = plt.subplots(nhigh, nwide, figsize=(nwide+fig_width, nhigh*fig_height))
fig.suptitle('Rows are')

for i,r in enumerate(runs):
    filename="20210708_"+r+"_data_with_WFE_m1Pos_centroids.csv"
    df[r] = pd.read_csv("data/"+filename, index_col=0)
    print(filename)
    df[r].index=pd.to_datetime(df[r].index)
    
    xvals = (df[r].m1_x_pos) # um
    yvals = df[r].el
    row1[0].plot(xvals, yvals, sym[i])
    row1[0].set_ylabel('Elevation [deg]')
    row1[0].set_xlabel('M1 X-position [mm]')

    xvals=df[r].y
    row1[1].plot(xvals,yvals, sym[i])
    row1[1].set_xlabel('Hexapod Y-position [mm]')

    xvals = (df[r].m1_tip) # um   - Tip should mean azimuth motion
    row1[2].plot(xvals, yvals, sym[i])
    row1[2].set_ylabel('Elevation [deg]')
    row1[2].set_xlabel('M1 tip [arcsec]')

    xvals = (df[r].azShift) # um
    row1[3].plot(xvals, yvals, sym[i])
    row1[3].set_ylabel('Elevation [deg]')
    row1[3].set_xlabel('Azimuth Image Offset [arcsec]')

    xvals = (df[r].m1_y_pos) # mm
    row2[0].plot(xvals, yvals, sym[i])
    row2[0].set_ylabel('Elevation [deg]')
    row2[0].set_xlabel('M1 X-position [mm]')

    xvals=df[r].x
    row2[1].plot(xvals,yvals, sym[i])
    row2[1].set_xlabel('Hexapod X-position [mm]')

    xvals = (df[r].m1_tilt) # um
    row2[2].plot(xvals, yvals, sym[i])
    row2[2].set_ylabel('Elevation [deg]')
    row2[2].set_xlabel('M1 tilt [arcsec]')

    xvals = (df[r].elShift) # um
    row2[3].plot(xvals, yvals, sym[i])
    row2[3].set_ylabel('Elevation [deg]')
    row2[3].set_xlabel('Elevation Image Offset [arcsec]')

    xvals = (df[r].m1_piston) # mm
    row3[0].plot(xvals, yvals, sym[i])
    row3[0].set_ylabel('Elevation [deg]')
    row3[0].set_xlabel('M1 Z-position [mm]')

    xvals=df[r].z
    row3[1].plot(xvals,yvals, sym[i])
    row3[1].set_xlabel('Hexapod Z-position [mm]')